In [3]:
import numpy as np
import pandas as pd

In [1]:
class BacktestExecutor():
    
    # Constructor stores reference to the portfolio_db
    def __init__(self,portfolio_db):
        self.portfolio_db = portfolio_db
    
    # Executes the passed open_trades_df using the information in the passed price_df
    def execute_opens(self,open_trades_df,price_df):
        # We dont' want to mess up the columns of the passed trades_df, so we save their names and order
        original_columns = open_trades_df.columns
        
        # Need security_ids to have type object, change them if they don't
        if open_trades_df.loc[:,'security_id'].dtype != object:            
            open_trades_df['security_id'] = open_trades_df.loc[:,'security_id'].astype(object)            
            
        if price_df.loc[:,'security_id'].dtype != object:            
            price_df['security_id'] = price_df.loc[:,'security_id'].astype(object)

        # Assume we can execute at close with no slippage, so all we need from price_df is 'prc'
        open_trades_df = open_trades_df.merge(price_df.loc[:,['security_id','prc']],on='security_id')
        open_trades_df['open_average_price'] = open_trades_df['prc']
        open_trades_df = open_trades_df[original_columns]
                
        # now update portfolio_db by calling open_trades
        self.portfolio_db.open_trades(open_trades_df)
        
    # Executes the passed close_trades_df using the information in the passed price_df
    def execute_closes(self,close_trades_df,price_df):
        original_columns = close_trades_df.columns

        # Need security_ids to have type object, change them if they don't
        if close_trades_df['security_id'].dtype != object:            
            close_trades_df['security_id'] = close_trades_df['security_id'].astype(object)
            
        if price_df['security_id'].dtype != object:            
            price_df['security_id'] = price_df['security_id'].astype(object)

        # for now, assume we can execute at close with no slippage, so all we need from price_df is 'prc'
        close_trades_df = close_trades_df.merge(price_df.loc[:,['security_id','prc']],on='security_id')
        close_trades_df['close_average_price'] = close_trades_df['prc']
        close_trades_df = close_trades_df[original_columns]
            
        # now update portfolio_db by calling close_trades
        self.portfolio_db.close_trades(close_trades_df)